# What's new in scippneutron

This page highlights feature additions and discusses major changes from recent releases.
For a full list of changes see the [Release Notes](https://scipp.github.io/scippneutron/about/release-notes.html).

In [ ]:
import numpy as np
import scipp as sc
import scippneutron as scn

## File loading

<div class="alert alert-info">

**New in 0.3**

- `load_nexus` now loads the UB-matrix and orientation matrix if found in a Nexus file.
- `load_nexus` now also loads pulse times for event data.

</div>

## Instrument View

<div class="alert alert-info">

**New in 0.3**

Additional components can be displayed using a range of shapes and related options to embellish the 3D instrument plot.

</div>

Full details are described in the [Instrument View](../user-guide/instrument-view.ipynb) documentation.
Example:

In [ ]:
da = scn.load(scn.data.get_path('PG3_4844_event.nxs'))

In [ ]:
sample_settings = {'center': da.meta['sample_position'], 'color': '#000000', 'wireframe': True,
                   'size': sc.scalar(0.2, unit='m'), 'type': 'box'}
source_settings = {'center': da.meta['source_position'], 'color': '#FFC133',
                   'size': sc.vector(value=[1, 2, 1], unit='m'), 'type': 'cylinder'}
scn.instrument_view(da, components={'sample': sample_settings, 'source': source_settings})

## Coordinate Transformations
(a.k.a 'Unit Conversions')

<div class="alert alert-info">

**New in 0.4**

A new API based on [sc.transform_coords](https://scipp.github.io/generated/functions/scipp.transform_coords.html#scipp.transform_coords) that allows for more flexibility than [scn.convert](../generated/scippneutron.convert.rst#scippneutron.convert).

</div>

Full details are described in the [Coordinate Transformations](../user-guide/coordinate-transformations.ipynb) documentation.

The new API uses graphs that describe how coordinates can be computed from each other.
Scippneutron comes with a few predefined graphs for common use cases, see [scippneutron.conversion.graph](../generated/modules/scippneutron.conversion.graph.rst).
Those graphs can be customized to, for example, account for gravity.
This can be done by changing how $2\theta$ is computed.

In [ ]:
from scipp.constants import m_n, h


def two_theta(gravity, wavelength, incident_beam, scattered_beam):
    # Arbitrary internal convention: beam=z, gravity=y
    g = sc.norm(gravity)
    L2 = sc.norm(scattered_beam)
    y = sc.dot(scattered_beam, gravity) / g
    n = sc.cross(incident_beam, gravity)
    n /= sc.norm(n)
    x = sc.dot(scattered_beam, n)
    wavelength = sc.to_unit(wavelength, "m", copy=False)
    drop = g * m_n ** 2 / (2 * h ** 2) * wavelength ** 2 * L2 ** 2
    return sc.asin(sc.sqrt(x ** 2 + (y + drop) ** 2) / L2)

Now construct a default graph for computing $Q$ and replace the implementation of `two_theta`:

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic_Q

q_with_gravity = {**beamline(scatter=True), **elastic_Q("tof")}
q_with_gravity["two_theta"] = two_theta

Resulting in the following conversion graph.

In [ ]:
sc.show_graph(q_with_gravity, simplified=True)

Which can be used with [sc.transform_coords](https://scipp.github.io/generated/functions/scipp.transform_coords.html#scipp.transform_coords) to convert from time-of-flight to $Q$:

In [ ]:
from scipp.constants import g

da = scn.data.tutorial_dense_data()
da.coords["gravity"] = sc.vector(value=[0, -1, 0]) * g
da_gravity = da.transform_coords("Q", graph=q_with_gravity)
da_gravity